# Задание

Обучить 2 диалоговые модели (decoder-only и encoder-decoder) предсказывать ответ системы на запрос пользователя.

1. Выбрать подходящую модель https://huggingface.co/models.
2. Выбрать подходящий датасет https://huggingface.co/datasets.

Выполнить шаги 3-8 для decoder-only и для encoder-decoder модели.

3. Проверить работу модели в сценарии близком к выбранному датасету в zero-shot режиме.
4. Посчитать метрики bleu и meteor исходной модели на валидационном сабсете выбранного датасета.
5. Дообучить модель на обучающем сабсете выбранного датасета.
6. Проверить работу модели в сценарии близком к выбранному датасету.
7. Посчитать метрики bleu и meteor обученной модели на валидационном сабсете выбранного датасета.
8. Сравнить метрики обученной и исходной моделей.

# Imports

In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
import datasets
from sklearn.model_selection import train_test_split
import pandas as pd
from torch.utils.data import TensorDataset
import evaluate
from torch.utils.data import DataLoader
import re
from torchmetrics.text import BLEUScore
from transformers import AutoTokenizer, AutoModelForCausalLM


# Dataset

In [2]:
ds = datasets.load_dataset("igorktech/anekdots_dialogs")
ds = ds.remove_columns(['original','date', 'downvote', 'total_votes', 'upvote', 'hash', 'alpha_frac', 'LDR', 'days_since_publication', 'time_decay', 'LDR_time_decay'])
ds

DatasetDict({
    train: Dataset({
        features: ['parsed', 'total_mark'],
        num_rows: 100834
    })
})

In [3]:
ds["train"]["parsed"][:2]

[{'turns': [{'speaker_name': '', 'phrase': 'Вы в цирк пойдете?'},
   {'speaker_name': '', 'phrase': 'Нет, я новости смотрел.'}]},
 {'turns': [{'speaker_name': '', 'phrase': 'Моряки не плавают, а ходят.'},
   {'speaker_name': '',
    'phrase': 'Нет, по воде ходил только Иисуc Христос.'}]},


In [4]:
filtered_ds = [i for i in ds['train'] if len(i['parsed']['turns']) == 2]
print(len(filtered_ds))

48032


In [5]:
df = pd.DataFrame.from_dict(filtered_ds)
df.head()

,parsed,total_mark
0,"{'turns': [{'speaker_name': '', 'phrase': 'Вы ...",227
1,"{'turns': [{'speaker_name': '', 'phrase': 'Мор...",4
2,"{'turns': [{'speaker_name': '', 'phrase': 'Как...",-3
3,"{'turns': [{'speaker_name': '', 'phrase': 'Слы...",-5
4,"{'turns': [{'speaker_name': '', 'phrase': 'Что...",-9


In [6]:
top_df = df.sort_values('total_mark',ascending=False).iloc[0:30000]
top_df.head()

,parsed,total_mark
4673,"{'turns': [{'speaker_name': 'Дворецкий', 'phra...",4060
9514,"{'turns': [{'speaker_name': 'Ученики', 'phrase...",3145
28556,"{'turns': [{'speaker_name': 'Москвичка', 'phra...",3061
5395,"{'turns': [{'speaker_name': 'Водитель', 'phras...",2376
38011,"{'turns': [{'speaker_name': '', 'phrase': 'У в...",2295


In [7]:
top_df["parsed"][0]

{'turns': [{'speaker_name': '', 'phrase': 'Вы в цирк пойдете?'},
  {'speaker_name': '', 'phrase': 'Нет, я новости смотрел.'}]}

# Decoder only

## Model

In [8]:
model_path = 'ai-forever/rugpt3small_based_on_gpt2'

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

## Test run

In [10]:
text = 'Сколько человек нужно, чтобы закрутить лампочку?'
inputs = tokenizer(text, return_tensors='pt')

In [11]:
with torch.no_grad():
    # https://huggingface.co/docs/transformers/main/en/main_classes/text_generation
    hypotheses = model.generate(
        **inputs,
        temperature=0.9,
        do_sample=True,  # sampling or greedy decoding
        # (at each decoding step selects the token with the highest prob without considering the impact on future tokens)
        top_p=0.7,
        num_return_sequences=3,
        repetition_penalty=2.5,  # https://arxiv.org/pdf/1909.05858.pdf
        max_length=32,
    )

In [12]:
for h in hypotheses:
    print(tokenizer.decode(h, skip_special_tokens=True), end='\n\n')

Сколько человек нужно, чтобы закрутить лампочку?
Для начала определитесь с тем в какой квартире проводите свет. Есть ли у вас место для ночника

Сколько человек нужно, чтобы закрутить лампочку?
А я думаю что не больше 10. И это если лампочки установлены в помещении или на улице - и то

Сколько человек нужно, чтобы закрутить лампочку?
У меня есть 3 лампы накаливания. А вообще я считаю что лампочки надо крутиться по часовой стрел



## Data Prep

In [13]:
def tokenize_data(tokenizer, data, column_name, max_length=128):
    source_texts = data[column_name].tolist()
    tokens = tokenizer(
        source_texts,
        max_length=max_length,
        padding='max_length',
        padding_side="right",
        truncation=True,
        add_special_tokens=True,
        return_tensors='pt',
    )
    return tokens

In [14]:
train, other = train_test_split(top_df, test_size=0.2, random_state=42)
test, valid = train_test_split(other, test_size=0.5, random_state=42)

In [15]:
train['text'] = train.apply(lambda x: '<s> Первый: ' + x['parsed']['turns'][0]['phrase'] + ' Второй: ' + x['parsed']['turns'][1]['phrase'] + '</s>', axis=1)
valid['text'] = valid.apply(lambda x: '<s> Первый: ' + x['parsed']['turns'][0]['phrase'] + ' Второй: ' + x['parsed']['turns'][1]['phrase'] + '</s>', axis=1)
test['text'] = test.apply(lambda x: '<s> Первый: ' + x['parsed']['turns'][0]['phrase'] + ' Второй: ' + x['parsed']['turns'][1]['phrase'] + '</s>', axis=1)

In [16]:
valid['text'].iloc[1]

'<s> Первый: блядь, кто ни будь может объяснить, почему у нас  не регистрируют лекарства, которые необходимы детям,  ежедневно показываемым по телевизору с просьбой к населению скинутся  на эти лекарства? Второй: ждут, наверное, когда потерявшее терпение население скинется в бюджет на регистрацию этих лекарств...</s>'

In [17]:
valid["parsed"].head(10)

9045     {'turns': [{'speaker_name': '', 'phrase': 'Про...
34853    {'turns': [{'speaker_name': '', 'phrase': 'бля...
45580    {'turns': [{'speaker_name': '', 'phrase': 'А г...
20927    {'turns': [{'speaker_name': 'Пациент', 'phrase...
32292    {'turns': [{'speaker_name': '', 'phrase': 'Слы...
35362    {'turns': [{'speaker_name': '', 'phrase': 'Бэр...
22546    {'turns': [{'speaker_name': '', 'phrase': 'Дол...
8898     {'turns': [{'speaker_name': '', 'phrase': 'А т...
4805     {'turns': [{'speaker_name': '', 'phrase': 'От ...
19228    {'turns': [{'speaker_name': 'Учитель', 'phrase...
Name: parsed, dtype: object

In [18]:
train_tokens = tokenize_data(tokenizer, train, column_name='text')
val_tokens = tokenize_data(tokenizer, valid, column_name='text')
test_tokens = tokenize_data(tokenizer, test, column_name='text')

In [19]:
train_tokens['input_ids'][125]

tensor([    1,  7200,    30,   439,  2006,   282,  1175,  1003,   478, 35959,
          308,   414,   318,  9519,  7409,    30, 16345, 14113, 43508,     5,
            2,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [20]:
train_dataset = TensorDataset(
    train_tokens['input_ids'],
    train_tokens['attention_mask'],
)

val_dataset = TensorDataset(
    val_tokens['input_ids'],
    val_tokens['attention_mask'],
)

test_dataset = TensorDataset(
    test_tokens['input_ids'],
    test_tokens['attention_mask'],
)

In [21]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

## Fine-Tuning

In [22]:
device = 'mps' if torch.backends.mps.is_built() else 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

cuda


In [23]:
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50264, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50264, bias=False)
)

In [24]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

In [25]:
from torch.utils.data import DataLoader
from tqdm import tqdm


def train_epoch(
        model,
        loader: DataLoader,
        epoch,
        num_epochs,
        optimizer,
        mode,
):
    total_loss = 0

    for x_input_ids, x_attention_mask in tqdm(loader, desc=f'{mode} epoch {epoch}/{num_epochs}...'):
        if mode == 'Training':
            optimizer.zero_grad()

        x_input_ids = x_input_ids.to(device)
        x_attention_mask = x_attention_mask.to(device)

        outputs = model(
            input_ids=x_input_ids,
            attention_mask=x_attention_mask,
            labels=x_input_ids,
            return_dict=True,
        )
        outputs_loss = outputs.loss
        total_loss += outputs_loss.item()

        if mode == 'Training':
            outputs_loss.backward()
            optimizer.step()

    loss = total_loss / len(loader)
    print(f'{mode} epoch {epoch + 1}/{num_epochs}: {mode} Loss: {loss:.4f}')

In [26]:
def train(
        model,
        train_loader,
        val_loader,
        optimizer,
        num_epochs=5,
):
    for epoch in range(num_epochs):
        model.train()

        train_epoch(model, train_loader, epoch, num_epochs, optimizer, mode='Training')

        model.eval()

        with torch.no_grad():
            train_epoch(model, val_loader, epoch, num_epochs, optimizer, mode='Validating')

In [27]:
train(model, train_loader, val_loader, optimizer, num_epochs=4)

Training epoch 0/4...:   0%|          | 0/3000 [00:00<?, ?it/s]c:\Users\user\miniconda3\Lib\site-packages\transformers\models\gpt2\modeling_gpt2.py:545: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Training epoch 0/4...: 100%|██████████| 3000/3000 [09:01<00:00,  5.54it/s]


Training epoch 1/4: Training Loss: 0.9524


Validating epoch 0/4...: 100%|██████████| 375/375 [00:17<00:00, 21.15it/s]


Validating epoch 1/4: Validating Loss: 0.7873


Training epoch 1/4...: 100%|██████████| 3000/3000 [09:03<00:00,  5.52it/s]


...

Validating epoch 3/4: Validating Loss: 0.7820


Training epoch 3/4...: 100%|██████████| 3000/3000 [08:56<00:00,  5.59it/s]


Training epoch 4/4: Training Loss: 0.7255


Validating epoch 3/4...: 100%|██████████| 375/375 [00:17<00:00, 21.34it/s]

Validating epoch 4/4: Validating Loss: 0.7865


In [28]:
text = '<s> Первый: Сколько человек нужно, чтобы закрутить лампочку? Второй: '
# inputs = tokenizer(text, return_tensors='pt',padding_side="right",)
inputs = tokenizer(
        text,
        add_special_tokens=True,
        return_tensors='pt',    )

In [29]:
inputs['input_ids'] = inputs['input_ids'].cuda()
inputs['attention_mask'] = inputs['attention_mask'].cuda()

In [30]:
inputs['input_ids']

tensor([[    1,  7200,    30,  7883,  1135,  1513,    16,   753, 30629,  1938,
          7721, 39778,    35,  7409,    30,   225]], device='cuda:0')

In [31]:
with torch.no_grad():
    # https://huggingface.co/docs/transformers/main/en/main_classes/text_generation
    hypotheses = model.generate(
        **inputs,
        temperature=0.8,
        do_sample=True,  # sampling or greedy decoding
        # (at each decoding step selects the token with the highest prob without considering the impact on future tokens)
        top_p=0.7,
        num_return_sequences=3,
        repetition_penalty=2.5,  # https://arxiv.org/pdf/1909.05858.pdf
        max_new_tokens=128,
    )

In [32]:
for h in hypotheses:
    print()
    print(tokenizer.decode(h, skip_special_tokens=True))


 Первый: Сколько человек нужно, чтобы закрутить лампочку? Второй:  Пять.

 Первый: Сколько человек нужно, чтобы закрутить лампочку? Второй:  Достаточно одного.

 Первый: Сколько человек нужно, чтобы закрутить лампочку? Второй:  Не знаю. Я не специалист по этой части...


## Evaluating BLEU on val dataset

In [33]:
def generate(
        model,
        inputs,
        num_return_sequences=5,
):
    return model.generate(
        **inputs,
        temperature=0.9,
        do_sample=True,
        top_p=0.7,
        num_return_sequences=num_return_sequences,
        repetition_penalty=2.5,
        max_length=128,
    )

In [34]:
initial_model = AutoModelForCausalLM.from_pretrained(model_path)
initial_model = initial_model.to(device)

In [35]:
valid["parsed"].iloc[1]["turns"][0]

{'speaker_name': '',
 'phrase': 'блядь, кто ни будь может объяснить, почему у нас  не регистрируют лекарства, которые необходимы детям,  ежедневно показываемым по телевизору с просьбой к населению скинутся  на эти лекарства?'}

In [36]:
valid["parsed"].head(10)

9045     {'turns': [{'speaker_name': '', 'phrase': 'Про...
34853    {'turns': [{'speaker_name': '', 'phrase': 'бля...
45580    {'turns': [{'speaker_name': '', 'phrase': 'А г...
20927    {'turns': [{'speaker_name': 'Пациент', 'phrase...
32292    {'turns': [{'speaker_name': '', 'phrase': 'Слы...
35362    {'turns': [{'speaker_name': '', 'phrase': 'Бэр...
22546    {'turns': [{'speaker_name': '', 'phrase': 'Дол...
8898     {'turns': [{'speaker_name': '', 'phrase': 'А т...
4805     {'turns': [{'speaker_name': '', 'phrase': 'От ...
19228    {'turns': [{'speaker_name': 'Учитель', 'phrase...
Name: parsed, dtype: object

In [37]:
bleu = BLEUScore(n_gram=1)
avg_bleu_score = 0.
avg_blue_score_org = 0.

cnt = 100
with torch.no_grad():
    for _, row in tqdm(valid["parsed"].head(cnt).items(), total=cnt):
        source = 'Первый: ' + row['turns'][0]['phrase'] + ' Второй: '
        target = 'Первый: ' + row['turns'][0]['phrase'] + ' Второй: ' + row['turns'][1]['phrase']  

        inputs = tokenizer(source, return_tensors='pt')
        for key in inputs.keys():
            inputs[key] = inputs[key].to(device)

        hypotheses = generate(model, inputs, num_return_sequences=1)
        pred = tokenizer.decode(hypotheses[0], skip_special_tokens=True)

        # shrinked_target = re.split('\.|\?', target)[0]
        # shrinked_pred = re.split('\.|\?', pred)[0]

        # metric = bleu(shrinked_pred, shrinked_target)
        metric = bleu(target, pred)
        avg_bleu_score += metric.item()


        hypotheses = generate(initial_model, inputs, num_return_sequences=1)
        pred = tokenizer.decode(hypotheses[0], skip_special_tokens=True)

        # shrinked_target = re.split('\.|\?', target)[0]
        # shrinked_pred = re.split('\.|\?', pred)[0]

        # metric = bleu(shrinked_pred, shrinked_target)
        metric = bleu(target, pred)
        avg_blue_score_org += metric.item()

print()
print('avg bleu:', avg_bleu_score * 1. / cnt)
print('avg initial bleu:', avg_blue_score_org * 1. / cnt)

100%|██████████| 100/100 [02:52<00:00,  1.72s/it]


avg bleu: 0.7447457104921341
avg initial bleu: 0.6172576893866062


In [38]:
target

'Первый: Мужики, у нас есть повод выпить? Второй: Повод - да! Выпить - нет!'

In [39]:
pred

'Первый: Мужики, у нас есть повод выпить? Второй: ))))\nЯ вот тоже... но я не пью. А то что сейчас все пьют и говорят - это хорошо!))))) ) Вот так-то... Но когда мы с женой выпили по рюмочке водки за здоровье мужа в этом году (с мужем), он сразу сказал мне "давай!" :-) И если бы она сказала еще раз на том же месте где была вчера или позавчера...а может быть вообще ничего этого тогда сказать было нельзя!!!А сегодня муж пьет водку как пьяный! Я его спрашиваю зачем? Он говорит а ты куда собрался??!! Да'

## Evaluating METEOR on val dataset

In [40]:
meteor = evaluate.load('meteor')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [41]:
avg_meteor_score = 0.
initial_meteor_score = 0.

cnt = 100
with torch.no_grad():
    for _, row in tqdm(valid["parsed"].head(cnt).items(), total=cnt):
        source = 'Первый: ' + row['turns'][0]['phrase'] + ' Второй: '
        target = 'Первый: ' + row['turns'][0]['phrase'] + ' Второй: ' + row['turns'][1]['phrase']  

        inputs = tokenizer(source, return_tensors='pt')
        for key in inputs.keys():
            inputs[key] = inputs[key].to(device)
        hypotheses = generate(model, inputs, num_return_sequences=1)
        not_fine_tuned_hypotheses = generate(initial_model, inputs, num_return_sequences=1)

        pred = tokenizer.decode(hypotheses[0], skip_special_tokens=True)
        not_fine_tuned_preds = tokenizer.decode(not_fine_tuned_hypotheses[0], skip_special_tokens=True)



        # shrinked_target = re.split('\.|\?', target)[0]
        # shrinked_pred = re.split('\.|\?', pred)[0]
        # shinked_initial_preds = re.split('\.|\?', not_fine_tuned_preds)[0]
        

        # metric = meteor.compute(predictions=[shrinked_pred], references=[shrinked_target])
        # initial_metric = meteor.compute(predictions=[shinked_initial_preds], references=[shrinked_target])

        metric = meteor.compute(predictions=[pred], references=[target])
        initial_metric = meteor.compute(predictions=[not_fine_tuned_preds], references=[target])

        avg_meteor_score += metric['meteor']
        initial_meteor_score += initial_metric['meteor']

print()
print('avg meteor:', avg_meteor_score * 1. / cnt)
print('avg initial meteor:', initial_meteor_score * 1. / cnt)

100%|██████████| 100/100 [03:00<00:00,  1.81s/it]


avg meteor: 0.6292897793398825
avg initial meteor: 0.4643352057585915


In [42]:
results_dict = {}
results_dict.update({
    "Decoder_only_initial": {"METEOR": initial_meteor_score * 1. / cnt, "BLEU":avg_blue_score_org * 1. / cnt},
    "Decoder_only_tuned": {"METEOR": avg_meteor_score * 1. / cnt, "BLEU":avg_bleu_score * 1. / cnt}
})

# Итоги

In [43]:
pd.DataFrame.from_dict(results_dict)

,Decoder_only_initial,Decoder_only_tuned
METEOR,0.464335,0.629290
BLEU,0.617258,0.744746
